Variáveis Objetivo (Soja, milho, trigo e algodão)

In [1]:
# Importing the required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [29]:
class Crop:
    def __init__(self, id, name, link, file_name, comm_file):
        self.id = id
        self.name = name
        self.link = link
        self.file_name = file_name
        self.comm_file = comm_file

    def web_scrap(self):
        data = requests.get(self.link).text
        
        # Creating BeautifulSoup object
        soup = BeautifulSoup(data, 'html.parser')

        # Creating list with all tables
        tables = soup.find_all('table')

        #  Looking for the table with the classes 'wikitable' and 'sortable'
        table = soup.find('table', class_='table table-main orange')

        # Defining of the dataframe
        df = pd.DataFrame(columns=['Data', 'Estadual'])

        dates = []
        states = []
        countrys = []
        # Collecting Ddata
        for row in table.tbody.find_all('tr'):    
            # Find all data for each column
            head = row.find_all('th')
            columns = row.find_all('td')
            
            if(head != []):
                date = head[0].text.strip()
                dates.append(date)

            
            if(columns !=[]):
                value1 = columns[0].text.strip()
                value2 = columns[1].text.strip()

                states.append(value1)
                countrys.append(value2)

        df = pd.DataFrame(list(zip(dates, states, countrys)),columns =['Data', 'Estadual', 'País'])
        df['Id'] = self.id
        df['Cultivo'] = self.name

        df.to_csv(self.file_name, sep=';', index=False)

    def add_comm(self):
        df_comm = pd.read_csv(self.comm_file, sep=',')
        df_cult = pd.read_csv(self.file_name, sep=';')

        df_comm['Data'] = df_comm['Data'].astype('datetime64[ns]')
        df_cult['Data'] = df_cult['Data'].astype('datetime64[ns]')

        df = df_cult.merge(df_comm, on="Data", how='right')
        df.to_csv(self.file_name, sep=';', index=False)


In [42]:

path = "data library/"
path_comm = "data library/Commodities/"

soja = Crop(0, "Soja", "https://www.agrolink.com.br/cotacoes/historico/mt/soja-em-grao-sc-60kg", 
            path+"Soja_v1.csv", path_comm+"Soja-Chicago.csv")
milho = Crop(1, "Milho", "https://www.agrolink.com.br/cotacoes/historico/pr/milho-seco-sc-60kg", 
            path+"Milho_v1.csv", path_comm+"Milho-Chicago.csv")
trigo = Crop(2, "Trigo", "https://www.agrolink.com.br/cotacoes/historico/pr/trigo-em-grao-nacional-sc-60kg", 
             path+"Trigo_v1.csv", path_comm+"Trigo-Chicago.csv")
algodao = Crop(3, "Algodão", "https://www.agrolink.com.br/cotacoes/historico/mt/algodao-em-pluma-15kg", 
               path+"Algodao_v1.csv", path_comm+"Algodao-NY.csv")

soja.web_scrap()
milho.web_scrap()
trigo.web_scrap()
algodao.web_scrap()

#soja.add_comm()

Adicionar valores das Commodities

In [60]:
df_cult = pd.read_csv(path+"Soja_v1.csv", sep=';', index_col=False)
df_comm = pd.read_csv(path_comm+"Soja-Chicago.csv", sep=',', index_col=False)

df_cult['Data'] = df_cult['Data'].astype('datetime64[ns]')
df_comm['Data'] = df_comm['Data'].astype('datetime64[ns]')

df = df_cult.merge(df_comm, on="Data", how='right')
df


,Data,Estadual,País,Id,Cultivo,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2024-01-09,NaN,NaN,NaN,NaN,"1.052,12","1.000,12","1.069,25","995,00","1,35M","7,14%"
1,2024-01-08,NaN,NaN,NaN,NaN,"982,00","1.030,00","1.045,00","936,25","167,33K","-4,52%"
2,2024-01-07,NaN,NaN,NaN,NaN,"1.028,50","1.148,25","1.191,50","1.020,50","466,67K","-10,60%"
3,2024-01-06,NaN,NaN,NaN,NaN,"1.150,50","1.204,25","1.205,50","1.146,25","2,32M","-4,40%"
4,2024-01-05,NaN,NaN,NaN,NaN,"1.203,50","1.161,00","1.256,50","1.156,25","1,83M","3,48%"
...,...,...,...,...,...,...,...,...,...,...,...
241,2004-01-08,NaN,NaN,NaN,NaN,"627,25","591,00","712,00","565,00","85,40K","4,63%"
242,2004-01-07,NaN,NaN,NaN,NaN,"599,50","892,00","1.035,00","599,00","250,91K","-32,87%"
243,2004-01-06,NaN,NaN,NaN,NaN,"893,00","828,00","940,00","0,00","656,34K","9,71%"
244,2004-01-05,NaN,NaN,NaN,NaN,"814,00","1.036,00","1.063,00","812,00","542,53K","-21,28%"


Bolsa de Chicago

In [5]:
url = "https://br.investing.com/commodities/us-wheat-historical-data"

data = requests.get(url).text

# Creating BeautifulSoup object
soup = BeautifulSoup(data, 'html.parser')

In [7]:
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

Classes of each table:
['datatable_table__DE_1_', 'relative-selector-v2_table-browser__TI4_F', 'datatable_table--border__XOKr2']
['freeze-column-w-1', 'w-full', 'overflow-x-auto', 'text-xs', 'leading-4']
['w-full', 'text-xs', 'leading-4']
['w-full', 'text-xs', 'leading-4', 'hidden']
['w-full', 'text-xs', 'leading-4', 'hidden']


In [8]:
table = soup.find('table', class_='freeze-column-w-1 w-full overflow-x-auto text-xs leading-4')

table

<table class="freeze-column-w-1 w-full overflow-x-auto text-xs leading-4"><thead class="relative after:absolute after:bottom-0 after:left-0 after:right-0 after:h-px after:bg-[#ECEDEF]"><tr class="h-[41px]"><th class="datatable-v2_cell__IwP1U sticky left-0 min-w-[106px] bg-white text-left font-semibold text-v2-black"><div class="datatable-v2_cell__wrapper__7O0wk !block md:!inline-flex"></div></th><th class="datatable-v2_cell__IwP1U text-v2-black font-semibold text-xs leading-4 tracking-[0.2px] text-right rtl:text-right w-[10%] min-w-[77px]"><div class="datatable-v2_cell__wrapper__7O0wk !block md:!inline-flex"></div></th><th class="datatable-v2_cell__IwP1U text-v2-black font-semibold text-xs leading-4 tracking-[0.2px] text-right rtl:text-right w-[10%] min-w-[77px]"><div class="datatable-v2_cell__wrapper__7O0wk !block md:!inline-flex"></div></th><th class="datatable-v2_cell__IwP1U text-v2-black font-semibold text-xs leading-4 tracking-[0.2px] text-right rtl:text-right w-[10%] min-w-[77px]

In [9]:
dates = []
states = []
countrys = []
# Collecting Ddata
for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    head = row.find_all('th')
    columns = row.find_all('td')
    
    if(columns !=[]):
        date = columns[0].text.strip()
        value1 = columns[1].text.strip()
        value2 = columns[2].text.strip()

        dates.append(date)
        states.append(value1)
        countrys.append(value2)

df = pd.DataFrame(list(zip(dates, states, countrys)),columns =['Data', 'Estadual', 'País'])

df.to_csv("texte.csv", sep=';', index=False)